# 報酬関数によるConvex MDPの実現

参考：
* [Reward is Enough for Convex MDPs](https://arxiv.org/abs/2106.00661)

強化学習では通常，次の問題を解きます：

$$
\mathrm{RL}: \quad \max _{d_\pi \in \mathcal{K}} \sum_{s, a} r(s, a) d_\pi(s, a)
$$

これは$d_\pi$について凸な最適化問題ですね．一方で，次のような一般的な凸最適化問題を考えることができます．

$$
\text { Convex MDP: } \quad \min _{d_\pi \in \mathcal{K}} f\left(d_\pi\right) \text {, }
$$

ここで$f: \mathcal{K}\to \mathbb{R}$は凸関数です．
この$f$を変えることで様々な別の問題を導出することができます．
例えば$f(d_\pi)=d_\pi \cdot \log(d_\pi)$とすると，これはreward free MDPと同じです．

このような一般のConvex MDPは実は報酬と方策側のMin-maxゲームとして解くことができます．

表記：Average-rewardを考えることもできますが，面倒なので割引ありのケースだけ考えます．
* $J_\pi^\gamma=(1-\gamma) \mathbb{E} \sum_{t=1}^{\infty} \gamma^t r_t$
* $d_\pi^\gamma(s, a)=(1-\gamma) \mathbb{E} \sum_{t=1}^{\infty} \gamma^t \mathbb{P}_\pi\left(s_t=s\right) \pi(s, a)$

とします．すると，
* $J_\pi=\sum_{s, a} r(s, a) d_\pi(s, a)$
* $\mathcal{K}_\gamma=\left\{d_\pi \mid d_\pi \geq 0, \quad \sum_a d_\pi(s, a)=(1-\gamma) d_0(s)+\gamma \sum_{s^{\prime}, a^{\prime}} P\left(s, s^{\prime}, a^{\prime}\right) d_\pi\left(s^{\prime}, a^{\prime}\right) \quad \forall s \in S\right\}$

が成立します．
Convex MDPは$\left(S, A, P, f, \gamma, d_0\right)$として定義されます．

## Convex MDPをRLで解く

凸共役を利用することでconvex MDPはRLを利用して解くことができます．
次のFenchel conjugateを思い出しましょう．

---

**Fenchel conjugate**

$f: \mathbb{R}^n \rightarrow \mathbb{R} \cup\{-\infty, \infty\}$に対して，
$$
f^*(x):=\sup _y x \cdot y-f(y)
$$
のことを$f$のFenchel conjugateと呼ぶ．

$f^*$は常に凸関数であり，また，$f^{**}$であることに注意しましょう．

---

これを使うと，

$$
f^{\mathrm{OPT}}=\min _{d_\pi \in \mathcal{K}} f\left(d_\pi\right)=\min _{d_\pi \in \mathcal{K}} \max _{\lambda \in \Lambda}\left(\lambda \cdot d_\pi-f^*(\lambda)\right)=\max _{\lambda \in \Lambda} \min _{d_\pi \in \mathcal{K}}\left(\lambda \cdot d_\pi-f^*(\lambda)\right)
$$

として表すことができます．
ここで$\Lambda$は$\left\{\partial f\left(d_\pi\right) \mid d_\pi \in \mathcal{K}\right\}$の閉包であり，凸集合です．

---

**$\Lambda$についての補足**

$f$が微分可能である場合，$\sup _y x \cdot y-f(y)$を与えるどの$y^*$も$x=\nabla f(y^*)$を満たすことを思い出しましょう．（[CVX_convex_functions.ipynb](CVX_convex_functions.ipynb)参照）

また，その閉包が凸集合であることの証明は[On Frank-Wolfe and Equilibrium Computation](https://papers.nips.cc/paper_files/paper/2017/hash/7371364b3d72ac9a3ed8638e6f0be2c9-Abstract.html)のTheorem 4を参照してください．

---

また，最後にmin-maxがひっくり返せるのは$\lambda$と$d_\pi$について凸であるためですね．
さて，次のラグランジアンを定義しましょう．
$$
\mathcal{L}\left(d_\pi, \lambda\right):=\lambda \cdot d_\pi-f^*(\lambda)
$$
固定された$\lambda$に対して，$\mathcal{L}$の最小化は通常のRLで実現されます．よって，もし最適な$\lambda^*$があれば最適な$d_\pi^*$が手に入るような気がしますが，次のように，それができない場合があります．


---

$$
\arg \max _{d_\pi \in \mathcal{K}} d_\pi \cdot r=\arg \min _{d_\pi \in \mathcal{K}} f\left(d_\pi\right)
$$

が成立するような報酬が存在しないようなMDPと凸関数$f$が存在する．

**証明**

CMDPで決定的な方策が最適にならない状況などを考えればすぐにわかりそうです．

---

単純に$\arg\max$を取ると最適解が得られない可能性がありますが，次のようにして全ての更新の平均を取ると，最終的に最適解に収束します．

$$
\begin{aligned}
& \text { for } k=1, \ldots, K \text { do } \\
& \quad \lambda^k=\operatorname{Alg}_\lambda\left(d_\pi^1, \ldots, d_\pi^{k-1} ; \mathcal{L}\right) \\
& \quad d_\pi^k=\operatorname{Alg}_\pi\left(-\lambda^k\right) \\
& \text { end for } \\
& \text { Return } \bar{d}_\pi^K=\frac{1}{K} \sum_{k=1}^K d_\pi^k, \bar{\lambda}^K=\frac{1}{K} \sum_{k=1}^K \lambda^k
\end{aligned}
$$

具体的な$\operatorname{Alg}$の設計は後で見ます．

---

**補足**

[ReLOAD: Reinforcement Learning with Optimistic Ascent-Descent for Last-Iterate Convergence in Constrained MDPs](https://arxiv.org/abs/2302.01275)を使うと，平均を取らずに最終的な最適解への収束を実現できます．

---

この$\operatorname{Alg}$はどのような性質を満たすべきでしょうか？
そのために，一連の流れをオンライン凸最適化として捉えます．

オンライン凸最適化では$K$個の凸な損失関数$\ell_1, \ell_2, \ldots, \ell_K: \mathcal{K} \rightarrow \mathbb{R}$を各ステップで最適化することを考えます．
最適化では，次の平均リグレットを最小化することを目的にします．
$$
\bar{R}_K:=\frac{1}{K}\left(\sum_{k=1}^K \ell_k\left(x_k\right)-\min _{x \in \mathcal{K}} \sum_{k=1}^K \ell_k(x)\right)
$$

上のアルゴリズムにおいては，コスト側のプレイヤ＝は$\ell_\lambda^k=-\mathcal{L}\left(\cdot, \lambda^k\right)$を，方策側のプレイヤーは$\ell_\pi^{\bar{k}}=\mathcal{L}\left(d_\pi^k, \cdot\right)$を扱います．それぞれのリグレットを$\bar{R}^\pi_K$，$\bar{R}^\lambda_K$としましょう．このとき，次が成立します．

---

$\bar{R}^\pi_K \leq \epsilon_K$と$\bar{R}^\lambda_K\leq \delta_K$が成立するとき，
$\bar{d}^K_\pi$と$\bar{\lambda}^K$は

$$
\min _{d_\pi \in \mathcal{K}} \mathcal{L}\left(d_\pi, \bar{\lambda}^K\right) \geq f^{O P T}-\epsilon_K-\delta_K \text { and } \max _{\lambda \in \Lambda} \mathcal{L}\left(\bar{d}_\pi^K, \lambda\right) \leq f^{O P T}+\epsilon_K+\delta_K
$$

を満たす．
また，

$$f^{\mathrm{OPT}} \leq f\left(\bar{d}_\pi^K\right)=\max _\lambda \mathcal{L}\left(\bar{d}_\pi^K, \lambda\right) \leq f^{\mathrm{OPT}}+\epsilon_K+\delta_K$$

なので，リグレットが小さいアルゴリズムを使うことでnear-optimalな$d_\pi$が得られます．

---


## いろいろなアルゴリズム

### コスト側のプレイヤー

---

**Follow the Leader (FTL)**

Follow the leaderアルゴリズムは次のようにして$\lambda$を更新します．

$$
\lambda^k=\arg \max _{\lambda \in \Lambda} \sum_{j=1}^{k-1} \mathcal{L}\left(d_\pi^j, \lambda\right)=\arg \max _{\lambda \in \Lambda}\left(\lambda \cdot \sum_{j=1}^{k-1} d_\pi^j-K f^*(\lambda)\right)=\nabla f\left(\bar{d}_\pi^{k-1}\right)
$$

このとき，FTLのリグレットは$\bar{R}_K \leq c / \sqrt{K}$であることが保証されます．

---

**Online Mirror Descent (OMD)**

OMDアルゴリズムは次のようにして$\lambda$を更新します．

$$
\lambda^k=\arg \max _{\lambda \in \Lambda}\left(\left(\lambda-\lambda^{k-1}\right) \cdot \nabla_\lambda \mathcal{L}\left(d_\pi^{k-1}, \lambda^{k-1}\right)+\alpha_k B_r\left(\lambda, \lambda^{k-1}\right)\right)
$$

ここで，$B_r$はBregmanダイバージェンスです．
OMDアルゴリズムのリグレットは$\bar{R}_K \leq c / \sqrt{K}$であることが保証されます．

---

### 方策側のプレイヤー

---

**Best Response**

一番単純なのは，$\lambda$が与えられたときのRLを解くことです．

$$
d_\pi^k=\arg \min _{d_\pi \in \mathcal{K}} \mathcal{L}_k\left(d_\pi, \lambda^k\right)=\arg \min _{d_\pi \in \mathcal{K}} d_\pi \cdot \lambda^k-f^*\left(\lambda^k\right)=\arg \max _{d_\pi \in \mathcal{K}} d_\pi \cdot\left(-\lambda^k\right),
$$

このときのリグレットは０になります．
また，ExactにMaxを解くのではなく，近似解を求めるのも良いです．（論文参照）

---

**Non-stationary RL**

毎回$\varepsilon$-最適な方策を求めるのは大変なので，１ステップで１回だけRLを更新することも考えられます．
論文ではUCRL2を使った状況について議論してるので，よく読んでみましょう．


## 制約がついた場合

次のように，制約がついたConvex MDPを考えることができます．

$$
\min _{d_\pi \in \mathcal{K}} f\left(d_\pi\right) \quad \text { subject to } \quad g_i\left(d_\pi\right) \leq 0, \quad i=1, \ldots m
$$

ここで，$f$と$g_i$は凸関数です．このラグランジュ形式を考えてみましょう．

$$
L\left(d_\pi, \mu\right)=f\left(d_\pi\right)+\sum_{i=1}^m \mu_i g_i\left(d_\pi\right)=\max _\nu\left(\nu \cdot d_\pi-f^*(\nu)\right)+\sum_{i=1}^m \mu_i \max _{v_i}\left(d_\pi v_i-g_i^*\left(v_i\right)\right) .
$$

であり，これはConvex-concaveの形式に見えませんが，$\zeta_i=\mu_i v_i$を導入すれば，

$$
L\left(d_\pi, \mu, \nu, \zeta_1, \ldots, \zeta_m\right)=\nu \cdot d_\pi-f^*(\nu)+\sum_{i=1}^m\left(d_\pi \zeta_i-\mu_i g_i^*\left(\zeta_i / \mu_i\right)\right)
$$

となり，これは$d_\pi$について凸かつ$\left(\nu, \mu, \zeta_1, \ldots, \zeta_m\right)$について凹です．

